# Feature Extraction

### 1. Data Cleaning & Pre-processing

**mon_standard.pkl**: This file contains data from "monitored" websites.

   - Class count: 95

   - Instance count: 19,000 (95 websites, each with 10 subpages which are non-index pages, observed 20 times each)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np

In [ ]:
# Load X1, X2, y

# Load X1
with open('/content/drive/My Drive/Machine Learning Project/CODES/X1.pkl', 'rb') as file:
    X1 = pickle.load(file)

# Load X2
with open('/content/drive/My Drive/Machine Learning Project/CODES/X2.pkl', 'rb') as file:
    X2 = pickle.load(file)

# Load y
with open('/content/drive/My Drive/Machine Learning Project/CODES/y.pkl', 'rb') as file:
    y = pickle.load(file)

In [ ]:
import random

num_samples_per_y = y.count(0)
num_sample = int(num_samples_per_y/2)

# Initialize lists to store sampled values
sampled_X1 = []
sampled_X2 = []
sampled_y = []

# Randomly sample num_sample values for each value of y
unique_y_values = set(y)
for val in unique_y_values:
    indices_for_y = [idx for idx, value in enumerate(y) if value == val]  # Find indices corresponding to each value of y
    sampled_indices = random.sample(indices_for_y, num_sample)  # Sample num_sample indices
    sampled_X1.extend([X1[i] for i in sampled_indices])
    sampled_X2.extend([X2[i] for i in sampled_indices])
    sampled_y.extend([y[i] for i in sampled_indices])

# Verify the lengths of sampled lists
print("Sampled X1 length:", len(sampled_X1))
print("Sampled X2 length:", len(sampled_X2))
print("Sampled y length:", len(sampled_y))


Sampled X1 length: 9500
Sampled X2 length: 9500
Sampled y length: 9500


In [ ]:
X1 = sampled_X1
X2 = sampled_X2
y = sampled_y

### 2a. Feature Extraction (Continuous Features)

1. Sequence of packet timestamps (X1)
2. Sequence of packet sizes (X2)
3. Sequence of cumulative packet sizes
4. Sequence of bursts



Continuous Feature 3: Sequence of Cumulative Packet Sizes

In [ ]:
# Compute the cumulative sum for each sequence
cumulative_sizes = [np.cumsum(seq) for seq in X2]

# Print the first 10 values of the cumulative sizes for the 1st element
print("First 10 values of cumulative sizes:")
print(cumulative_sizes[0][:10])

First 10 values of cumulative sizes:
[ -512 -1024  -512 -1024  -512 -1024  -512     0  -512 -1024]


Continuous Feature 4: Sequence of Bursts

In [ ]:
def calculate_bursts_and_durations(X1, X2):
    seq_of_bursts = []
    burst_duration = []

    for timestamps, sizes in zip(X1, X2):
        burst = []
        duration = []

        current_size = 0
        current_time = 0.0

        time_start = 0.0

        for time, size in zip(timestamps, sizes):
          if current_size == 0 or (size > 0 and current_size > 0) or (size < 0 and current_size < 0):
              current_size += size
              current_time = time - time_start
          else:
              burst.append(current_size)
              duration.append(current_time)
              current_size = size
              current_time = 0.0
              time_start = time

        burst.append(current_size)
        duration.append(time-time_start)
        seq_of_bursts.append(burst)
        burst_duration.append(duration)

    return burst_duration, seq_of_bursts

burst_duration, seq_of_bursts = calculate_bursts_and_durations(X1, X2)

print(burst_duration[0][:10])
print(seq_of_bursts[0][:10])

[0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1100000000000001, 0.0, 0.0, 0.0]
[-1024, 512, -512, 512, -512, 1024, -7168, 512, -512, 512]


In [ ]:
print(X1[0][:20])
print(X2[0][:20])

[0.0, 0.2, 0.2, 0.39, 1.0, 1.24, 1.24, 1.24, 1.46, 1.46, 1.46, 1.46, 1.46, 1.46, 1.46, 1.57, 1.57, 1.57, 1.57, 1.57]
[-512, -512, 512, -512, 512, -512, 512, 512, -512, -512, -512, -512, -512, -512, -512, -512, -512, -512, -512, -512]


### 2b. Feature Extraction (Categorical Features)


1. Number of incoming packets
2. Number of incoming packets as a fraction of the total number of packets
3. Number of outgoing packets
4. Number of outgoing packets as a fraction of the total number of packets
5. Total number of packets.
6. Packet rate
7. Incoming packet rate (client to server)
8. Outgoing packet rate (server to client)
9. Average time gap
10. Total incoming bytes
11. Total outgoing bytes
12. Total incoming bursts
13. Total outgoing bursts
14. Total bursts
15. Average Inter-arrival time for incoming packets per sample
16. Average inter-departure time for outgoing packets per sample
17. Total burst duration

In [ ]:
# 1. Number of incoming packets
incoming_packets = [sum(1 for size in size_seq if size > 0) for size_seq in X2]

# 2. Number of incoming packets as a fraction of the total number of packets
fraction_incoming_packets = [sum(1 for size in size_seq if size > 0) / len(size_seq) for size_seq in X2]

# 3. Number of outgoing packets
outgoing_packets = [sum(1 for size in size_seq if size < 0) for size_seq in X2]

# 4. Number of outgoing packets as a fraction of the total number of packets
fraction_outgoing_packets = [sum(1 for size in size_seq if size < 0) / len(size_seq) for size_seq in X2]

# Print first 10 values of the resulting arrays
print("Incoming Packets Array:")
print(incoming_packets[:10])

print("\nFraction of Incoming Packets Array:")
print(fraction_incoming_packets[:10])

print("\nOutgoing Packets Array:")
print(outgoing_packets[:10])

print("\nFraction of Outgoing Packets Array:")
print(fraction_outgoing_packets[:10])

Incoming Packets Array:
[336, 376, 153, 483, 175, 168, 480, 458, 153, 156]

Fraction of Incoming Packets Array:
[0.1601525262154433, 0.1536575398447078, 0.06860986547085202, 0.049401656949984656, 0.09526401741970604, 0.16247582205029013, 0.049039640375970577, 0.046854219948849106, 0.06790945406125166, 0.06770833333333333]

Outgoing Packets Array:
[1762, 2071, 2077, 9294, 1662, 866, 9308, 9317, 2100, 2148]

Fraction of Outgoing Packets Array:
[0.8398474737845567, 0.8463424601552922, 0.931390134529148, 0.9505983430500153, 0.904735982580294, 0.8375241779497099, 0.9509603596240295, 0.9531457800511509, 0.9320905459387483, 0.9322916666666666]


In [ ]:
# 5. Total number of packets
total_packets = [len(size_seq) for size_seq in X2]

# 6. Packet Rate: Calculate the rate of packet arrival for each sequence
packet_rate = [len(seq) / (max(seq) - min(seq)) if len(seq) > 1 else 0 for seq in X1]

# 7. Incoming packet rate (client to server)
incoming_packet_rate = [sum(1 for size in sizes if size > 0) / (max(seq) - min(seq)) if len(seq) > 1 else 0 for seq, sizes in zip(X1, X2)]

# 8. Outgoing packet rate (server to client)
outgoing_packet_rate = [sum(1 for size in sizes if size < 0) / (max(seq) - min(seq)) if len(seq) > 1 else 0 for seq, sizes in zip(X1, X2)]

print("Total Packets Array:")
print(total_packets[:10])

print("\nPacket Rate:")
print(packet_rate[:10])

print("\nIncoming Packet Rate:")
print(incoming_packet_rate[:10])

print("\nOutgoing Packet Rate:")
print(outgoing_packet_rate[:10])

Total Packets Array:
[2098, 2447, 2230, 9777, 1837, 1034, 9788, 9775, 2253, 2304]

Packet Rate:
[62.4962764372952, 72.7624145108534, 227.3190621814475, 588.2671480144404, 112.14896214896216, 103.7111334002006, 770.7086614173229, 481.0531496062992, 206.88705234159778, 177.64070932922127]

Incoming Packet Rate:
[10.00893655049151, 11.180493606898601, 15.596330275229358, 29.061371841155232, 10.683760683760685, 16.850551654964892, 37.795275590551185, 22.539370078740156, 14.049586776859503, 12.02775636083269]

Outgoing Packet Rate:
[52.487339886803696, 61.581920903954796, 211.72273190621814, 559.2057761732851, 101.46520146520147, 86.8605817452357, 732.9133858267717, 458.51377952755905, 192.8374655647383, 165.61295296838858]


In [ ]:
# 9. Average Time Gap: Calculate the average time gap for each sequence in X1
avg_time_gaps = []

for seq in X1:
    if len(seq) > 1:
        time_gaps_sum = sum(j - i for i, j in zip(seq, seq[1:]))
        avg_time_gap = time_gaps_sum / (len(seq) - 1)  # Subtract 1 because there are len(seq) - 1 time gaps
        avg_time_gaps.append(avg_time_gap)
    else:
        avg_time_gaps.append(0)

print("\nAverage Time Gaps:")
print(avg_time_gaps[:10])


Average Time Gaps:
[0.016008583690987125, 0.01374897792313982, 0.004401076716016151, 0.0017000818330605565, 0.00892156862745098, 0.009651500484027107, 0.0012976397261673647, 0.0020789850624104767, 0.004835701598579041, 0.005631784628745115]


In [ ]:
# 10. & 11. Total incoming and outgoing bytes
incoming_bytes = []
outgoing_bytes = []

for sample in X2:
    incoming = sum(size for size in sample if size > 0)
    outgoing = abs(sum(size for size in sample if size < 0))

    incoming_bytes.append(incoming)
    outgoing_bytes.append(outgoing)

# Print total incoming and outgoing bytes for the first 10 samples
print(f'Incoming Bytes: {incoming_bytes[:10]}')
print(f'Outgoing Bytes: {outgoing_bytes[:10]}')

Incoming Bytes: [172032, 192512, 78336, 247296, 89600, 86016, 245760, 234496, 78336, 79872]
Outgoing Bytes: [902144, 1060352, 1063424, 4758528, 850944, 443392, 4765696, 4770304, 1075200, 1099776]


In [ ]:
# 12. & 13. Number of incoming and outgoing burst

total_incoming_bursts = []
total_outgoing_bursts = []

# Calculate total number of incoming and outgoing bursts for all samples
for sample in seq_of_bursts:
  incoming_bursts = sum(1 for val in sample if val > 0)
  outgoing_bursts = sum(1 for val in sample if val < 0)

  total_incoming_bursts.append(incoming_bursts)
  total_outgoing_bursts.append(outgoing_bursts)

# 14. Calculate burst count for each sample
burst_count = [len(bursts) for bursts in seq_of_bursts]

# Print total incoming and outgoing bursts for first 10 samples
print(f"Total Incoming Bursts: {total_incoming_bursts[:10]}")
print(f"Total Outgoing Bursts: {total_outgoing_bursts[:10]}")
print(f"Burst Count: {burst_count[:10]}")



Total Incoming Bursts: [128, 127, 104, 349, 97, 65, 343, 348, 102, 100]
Total Outgoing Bursts: [129, 128, 104, 349, 97, 65, 343, 348, 102, 100]
Burst Count: [257, 255, 208, 698, 194, 130, 686, 696, 204, 200]


In [ ]:
# 15. Calculate average inter-arrival time for incoming packets per sample
avg_interarrival_times = []

for idx, (sample_packets, sample_directions) in enumerate(zip(X1, X2)):
    incoming_packet_times = []

    # Filter incoming packets based on positive direction values
    incoming_packet_times = [packet_time for packet_time, direction in zip(sample_packets, sample_directions) if direction > 0]

    if len(incoming_packet_times) <= 1:
        # If only one or no incoming packet in the sample, assign 0 average inter-arrival time
        avg_interarrival_times.append(0)
    else:
        # Calculate inter-arrival times between incoming packets
        interarrival_times = [incoming_packet_times[i + 1] - incoming_packet_times[i] for i in range(len(incoming_packet_times) - 1)]

        # Compute the average inter-arrival time for incoming packets
        avg_interarrival_time = sum(interarrival_times) / len(interarrival_times)
        avg_interarrival_times.append(avg_interarrival_time)

# Print average inter-arrival time for incoming packets per sample
print("Average inter-arrival time for incoming packets per sample:")
for i, avg_interarrival_time in enumerate(avg_interarrival_times[:10], start=1):
    print(f"Sample {i}: {avg_interarrival_time}")


Average inter-arrival time for incoming packets per sample:
Sample 1: 0.09919402985074625
Sample 2: 0.08888000000000001
Sample 3: 0.06388157894736843
Sample 4: 0.03417012448132781
Sample 5: 0.09304597701149424
Sample 6: 0.05832335329341318
Sample 7: 0.02622129436325678
Sample 8: 0.044157549234135667
Sample 9: 0.07078947368421053
Sample 10: 0.08245161290322581


In [ ]:
# 16. Calculate average inter-departure time for outgoing packets per sample
avg_interdepart_times = []

for idx, (sample_packets, sample_directions) in enumerate(zip(X1, X2)):
    outcoming_packet_times = []

    # Filter outgoing packets based on negative direction values
    outcoming_packet_times = [packet_time for packet_time, direction in zip(sample_packets, sample_directions) if direction < 0]

    if len(outcoming_packet_times) <= 1:
        # If only one or no outgoing packet in the sample, assign 0 average inter-depart time
        avg_interdepart_times.append(0)
    else:
        # Calculate inter-depart times between outgoing packets
        interdepart_times = [outcoming_packet_times[i + 1] - outcoming_packet_times[i] for i in range(len(outcoming_packet_times) - 1)]

        # Compute the average inter-depart time for outgoing packets
        avg_interdepart_time = sum(interdepart_times) / len(interdepart_times)
        avg_interdepart_times.append(avg_interdepart_time)

# Print average inter-depart time for outgoing packets per sample
print("Average inter-departure time for outgoing packets per sample:")
for i, avg_interdepart_time in enumerate(avg_interdepart_times[:10], start=1):
    print(f"Sample {i}: {avg_interdepart_time}")


Average inter-departure time for outgoing packets per sample:
Sample 1: 0.019063032367972743
Sample 2: 0.016246376811594205
Sample 3: 0.003044315992292871
Sample 4: 0.0017884429140213065
Sample 5: 0.008657435279951838
Sample 6: 0.011526011560693645
Sample 7: 0.0013591920060169766
Sample 8: 0.002181193645341348
Sample 9: 0.0034730824202000954
Sample 10: 0.00463903120633442


In [ ]:
# 17. Total burst duration
total_burst_duration = [sum(duration) for duration in burst_duration]
print(total_burst_duration[:10])

[8.799999999999997, 27.680000000000003, 5.560000000000002, 8.240000000000002, 6.780000000000001, 4.779999999999998, 6.039999999999999, 10.879999999999995, 5.219999999999999, 5.489999999999999]


### 3b. Model Testing

In [ ]:
# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Get a classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.50
              precision    recall  f1-score   support

           0       0.52      0.33      0.41        39
           1       0.58      0.42      0.49        26
           2       0.68      0.71      0.69        24
           3       0.64      0.58      0.61        24
           4       0.55      0.50      0.52        24
           5       0.39      0.50      0.44        26
           6       0.80      0.67      0.73        24
           7       0.64      0.44      0.52        32
           8       0.59      0.50      0.54        20
           9       0.37      0.53      0.43        19
          10       0.36      0.34      0.35        29
          11       0.28      0.29      0.28        28
          12       0.79      0.79      0.79        24
          13       0.12      0.12      0.12        25
          14       0.36      0.32      0.34        28
          15       0.63      0.44      0.52        27
          16       0.62      0.31      0.41        26
          17

#### 5b. Feature Selection

In [ ]:
# Get the names of the features
feature_names = ['incoming_packets', 'fraction_incoming_packets', 'outgoing_packets', 'fraction_outgoing_packets',
                 'total_packets', 'packet_rate', 'incoming_packet_rate', 'outgoing_packet_rate', 'avg_time_gaps',
                 'incoming_bytes', 'outgoing_bytes', 'total_incoming_bursts', 'total_outgoing_bursts', 'burst_count',
                 'avg_interarrival_times', 'avg_interdepart_times', 'total_burst_duration']

# Get feature importances
feature_importances = rf_clf.feature_importances_

# Create a dictionary to map feature names to importance scores
feature_importance_dict = dict(zip(feature_names, feature_importances))

# Print feature importance scores in descending order
print("Features importance scores in descending order")
sorted_feature_importances = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
for feature, importance in sorted_feature_importances:
    print(f"{feature}: {importance}")

Features importance scores in descending order
total_packets: 0.09569918513695211
incoming_packet_rate: 0.08458289793728216
fraction_incoming_packets: 0.08456070641541714
outgoing_packets: 0.08420321009490443
fraction_outgoing_packets: 0.08359740814237775
total_incoming_bursts: 0.08310617071777184
total_outgoing_bursts: 0.07416563256800875
outgoing_bytes: 0.07238941088505872
packet_rate: 0.07237102120813958
incoming_packets: 0.07223734367990736
incoming_bytes: 0.06672216062446722
outgoing_packet_rate: 0.06366847659346467
avg_time_gaps: 0.06269637599624832


In [ ]:
# Run the experiment multiple times
num_iterations = 10  # Change this based on your preference

# Initialize a dictionary to store the accuracy results for each top-k subset
accuracy_results = {k: [] for k in range(3, len(feature_names) + 1)}

for iteration in range(num_iterations):
    # Create subsets of X with top k features
    for k in range(3, len(feature_names) + 1):
        top_k_features = [feature for feature, _ in sorted_feature_importances[:k]]
        X_subset = X[:, [feature_names.index(feature) for feature in top_k_features]]

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.25, random_state=iteration)

        # Initialize a new model (you may want to retrain the model with the subset)
        rf_model_subset = RandomForestClassifier()

        # Train the model with the subset
        rf_model_subset.fit(X_train, y_train)

        # Make predictions on the testing set
        y_pred = rf_model_subset.predict(X_test)

        # Evaluate the model and store the accuracy
        accuracy = accuracy_score(y_test, y_pred)
        accuracy_results[k].append(accuracy)

# Calculate and print the average accuracy for each top-k subset
for k, accuracies in accuracy_results.items():
    average_accuracy = np.mean(accuracies)
    print(f"Top {k} Features Average Accuracy: {average_accuracy}")

In [ ]:
top_12_features = [feature for feature, importance in sorted_feature_importances[:12]]
print(top_12_features)


In [ ]:
X = [total_packets,
     outgoing_bytes,
     outgoing_packets,
     incoming_packets,
     fraction_outgoing_packets,
     fraction_incoming_packets,
     incoming_bytes,
     burst_count,
     incoming_packet_rate,
     total_incoming_bursts,
     total_outgoing_bursts,
     outgoing_packet_rate]

# Transpose the feature matrix X to have samples as rows and features as columns
X = np.array(X).T

y = np.array(y)